# Compute Functions and Jobs
__________________
The Compute module provides scalable compute capabilities to parallelize your computations. Compute enables users to package and execute your Python code within nodes hosted on Descartes Lab's cloud infrastructure. These nodes offer the ability to access imagery at extremely high rates of throughput to execute computations over nearly any spatio-temporal scale. 

Within the compute module, there are two primary objects that we work with:

 * [**Function:**](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) dynamically created, serverless functions containing user submitted, compiled code that you can submit many jobs to.
 * [**Job:**](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job) submitted request for a single invocation of a created Function. 

Let's start by importing the compute module and creating a test Function object. 

In [ ]:
from descarteslabs.compute import Function, Job

Next, we'll create a very basic `hello_world` function that imports a common geospatial Python package and returns a given argument.

In [ ]:
def hello(i):
    print(f"Hello, {i}")
    return f"hello {i}"

To create the Function object, we simply need to call `Function()` and specify the desired parameters. The recommended minimum parameters are your function, a name, and the image that will be used to build the Function environment.  Some common attributes used to better customize the performance of your scalable compute object include: 
 * cpus = Number of CPUs requested for a single job
 * memory = max memory available for each job
 * maximum_concurrency = max number of jobs to run in parallel
 * timeout = max length a job can run in seconds
 * retry_count = max number of times a job can be retried
 * requirements = list of Python dependencies required by this function

For other options, please see the [Compute documentation](https://docs.descarteslabs.com/guides/compute.html).

In [ ]:
print("creating function")
async_func = Function(
    hello,
    name="my-compute-hello2",
    image="python3.8:master_ec20a887a6a964cf",
    cpus=0.25,
    memory=512,
    maximum_concurrency=1,
    timeout=600,
    retry_count=0,
)
async_func.save()

It should take a few minutes for your Function to be built within your organization's batch compute environment on AWS. The code below will automatically check if your function has been built and is running every 10-seconds. 

In [ ]:
import time

# Check status of Function build
print(f"Waiting for Function to be built: {async_func.name}")
while async_func.status == "building":
    async_func.refresh()
    time.sleep(10)

# Print function status
print(f"Function built: {async_func.name} status = {async_func.status}")

Once the Function is built, we will test it by creating and submitting a Job. There are several ways you can submit jobs to a function:
 * `async_func(args)` - Pass arguments directly to the compute.Function
 * [`async_func.map()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.map) - Submit multiple jobs efficiently (discussed in more detail within the next example - "02_Create_Imagery.ipynb")

In [ ]:
# invoke the function
print("submitting a job")
job = async_func("Hello from my function!")

# print the job result and logs
print("waiting for the job to complete")
job.wait_for_completion()
print("Job results:", job.result)